### Get the User's Desired LLM

In [1]:
import langchain_test as lt 
import methods

model = lt.get_user_choice()
print("You have chosen", model.name)

Please input your choice of LLM
You have chosen ChatGPT


### Get User's Calendar

In [2]:
calendar = methods.get_calendar()
print(calendar)

user_query = lt.get_user_query()

{'name': 'Run', 'datetime_start': '2024-03-26T22:00:00-04:00', 'datetime_end': '2024-03-26T23:00:00-04:00', 'id': '2jdmu23fud2s2pfdt5qvg5gn5p'}
{'name': 'EECS 545: Machine Learning', 'datetime_start': '2024-03-27T09:00:00-04:00', 'datetime_end': '2024-03-27T10:30:00-04:00', 'id': '4ja5oo0s5vlt0mk97gaiv38hjj_20240327T130000Z'}
{'name': 'STAT 412: Intro to Prob Stat', 'datetime_start': '2024-03-28T10:00:00-04:00', 'datetime_end': '2024-03-28T11:30:00-04:00', 'id': '0s0rgk47g3el2sm6v7rk7f1f3e_20240328T140000Z'}
{'name': 'EECS 545: Machine Learning', 'datetime_start': '2024-04-01T09:00:00-04:00', 'datetime_end': '2024-04-01T10:30:00-04:00', 'id': '4ja5oo0s5vlt0mk97gaiv38hjj_20240401T130000Z'}
{'name': 'STATS 412: Exam 2', 'datetime_start': '2024-04-02', 'datetime_end': '2024-04-03', 'id': '4bkcu6gf7553aqqderhaaufmka'}
{'name': 'STAT 412: Intro to Prob Stat', 'datetime_start': '2024-04-02T10:00:00-04:00', 'datetime_end': '2024-04-02T11:30:00-04:00', 'id': '0s0rgk47g3el2sm6v7rk7f1f3e_2024040

### Set Up Prompts 

In [3]:
from langchain_core.prompts import ChatPromptTemplate

intent_prompt_template = ChatPromptTemplate.from_messages([
    ("human", "Identify the type of query that the following query is: {user_query}. Your response should be a single word that describes the intent of the query, and should be one of the following: createEvent, moveEvent, deleteEvent, checkSchedule, listEvents, updateEvent."),
])

update_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "append _format to the following user intent - {user_intent}"),
    ("system", "Only output the appended word. Your answer should be one word in the following format: userintent_format")
])

update_json_template = ChatPromptTemplate.from_messages([
    ("system", "The following is the user's intent - {user_query}"),
    ("system", "This is the user's calendar - {user_calendar}"),
    ("system", "Explain the update you are making. Create a valid JSON Object for to update Google Calendar based on the following instructions: {format}.")
])

parse_answer =  ChatPromptTemplate.from_messages([
    ("system", "Parse the following this answer ({answer}) for the JSON Object. Output only the JSON Object. Ensure its correctly formatted"),
])

### Build The Chain

In [4]:
from langchain_core.runnables import RunnablePassthrough
import formats

lcel_chain = intent_prompt_template | model.llm | {'user_intent':RunnablePassthrough()} |update_prompt_template | model.llm 
out = lcel_chain.invoke({"user_query":user_query})
command = out.content

lcel_chain_2 = update_json_template | model.llm 
out = lcel_chain_2.invoke({"user_query":user_query, "user_calendar":calendar, "time":methods.current_time, "format": formats.getFormat(command)[0]['parameters']['properties']})
print(out.content)

{
  "name": "Run",
  "date": "2024-03-26",
  "time": "10:00 PM",
  "new_date": "2024-03-27",
  "new_time": "3:00 PM"
}


### Format JSON


In [5]:
import json

json_response = json.loads(out.content)
print(json_response)
methods.moveEvent(json_response)

{'name': 'Run', 'date': '2024-03-26', 'time': '10:00 PM', 'new_date': '2024-03-27', 'new_time': '3:00 PM'}
{'kind': 'calendar#event', 'etag': '"3422997059954000"', 'id': '2jdmu23fud2s2pfdt5qvg5gn5p', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=MmpkbXUyM2Z1ZDJzMnBmZHQ1cXZnNWduNXAgdmFydW4uZDk5QG0', 'created': '2024-03-26T20:45:28.000Z', 'updated': '2024-03-27T00:15:29.977Z', 'summary': 'Run', 'creator': {'email': 'varun.d99@gmail.com', 'self': True}, 'organizer': {'email': 'varun.d99@gmail.com', 'self': True}, 'start': {'dateTime': '2024-03-26T22:00:00-04:00', 'timeZone': 'America/New_York'}, 'end': {'dateTime': '2024-03-26T23:00:00-04:00', 'timeZone': 'America/New_York'}, 'iCalUID': '2jdmu23fud2s2pfdt5qvg5gn5p@google.com', 'sequence': 6, 'reminders': {'useDefault': True}, 'eventType': 'default'}
Event "Run" moved to 2024-03-27 15:00:00-04:00
